<a href="https://colab.research.google.com/github/Adamantev/DataSet/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Untitled1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from google.colab import files
%matplotlib inline

In [ ]:
!git clone  https://github.com/Adamantev/DataSet.git


fatal: destination path 'DataSet' already exists and is not an empty directory.


In [ ]:
batch_size=32
image_size=(224, 224)

In [ ]:
train_dataset = image_dataset_from_directory('/content/DataSet/Training',
                                             subset='training',
                                             seed=42,
                                             validation_split=0.1,
                                             batch_size=batch_size,
                                             image_size=image_size)

Found 133 files belonging to 2 classes.
Using 120 files for training.


In [ ]:
validation_dataset = image_dataset_from_directory('/content/DataSet/Training',
                                             subset='validation',
                                             seed=42,
                                             validation_split=0.1,
                                             batch_size=batch_size,
                                             image_size=image_size)

Found 133 files belonging to 2 classes.
Using 13 files for validation.


In [ ]:
class_names = train_dataset.class_names
class_names

['Bad', 'Norm']

In [ ]:
test_dataset = image_dataset_from_directory('/content/DataSet/Test',
                                             batch_size=batch_size,
                                             image_size=image_size)

Found 10 files belonging to 2 classes.


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Создаем последовательную модель
model = Sequential()
# Сверточный слой
model.add(Conv2D(16, (3, 3), padding='same', 
                 input_shape=(224, 224, 3), activation='relu'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Сверточный слой
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Сверточный слой
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Полносвязная часть нейронной сети для классификации
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# Выходной слой, 2 нейрон по количеству классов
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset,
validation_data=validation_dataset,
epochs=20,
verbose=1,
callbacks = [tf.keras.callbacks.EarlyStopping(
monitor='val_accuracy', patience=2, verbose=1),
tf.keras.callbacks.ModelCheckpoint(
'model.h5', monitor='val_loss',
verbose=1, save_best_only=True)])

Epoch 1/20
4/4 [==============================] - ETA: 0s - loss: 2.4723e-05 - accuracy: 1.0000
Epoch 00001: val_loss improved from inf to 0.00013, saving model to model.h5
4/4 [==============================] - 2s 447ms/step - loss: 2.4723e-05 - accuracy: 1.0000 - val_loss: 1.2754e-04 - val_accuracy: 1.0000
Epoch 2/20
4/4 [==============================] - ETA: 0s - loss: 2.5746e-05 - accuracy: 1.0000
Epoch 00002: val_loss did not improve from 0.00013
4/4 [==============================] - 1s 132ms/step - loss: 2.5746e-05 - accuracy: 1.0000 - val_loss: 1.3749e-04 - val_accuracy: 1.0000
Epoch 3/20
4/4 [==============================] - ETA: 0s - loss: 2.2054e-05 - accuracy: 1.0000
Epoch 00003: val_loss did not improve from 0.00013
4/4 [==============================] - 1s 133ms/step - loss: 2.2054e-05 - accuracy: 1.0000 - val_loss: 1.4164e-04 - val_accuracy: 1.0000
Epoch 00003: early stopping


In [ ]:
model.load_weights('model.h5')
model.evaluate(test_dataset)

1/1 [==============================] - 0s 109ms/step - loss: 7.0044 - accuracy: 0.7000


[7.004427909851074, 0.699999988079071]

In [ ]:
print("Доля верных ответов на тестовых данных, в процентах:", round(scores[1] * 100, 4))


Доля верных ответов на тестовых данных, в процентах: 70.0
